In [1]:
import torch
from config import get_config, latest_weights_file_path
from train import get_model, get_ds, run_validation
config= get_config()
# config['datasource']='M:\Order to PC\CAD_Reconstruction\opus_books'

model_filename = latest_weights_file_path(config)


In [2]:
print(model_filename)

Aarif1430\english-to-hindi_weights\tmodel_27.pt


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

train_dataloader , val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])


Using the latest cached version of the dataset since Aarif1430/english-to-hindi couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\ms526\.cache\huggingface\datasets\Aarif1430___english-to-hindi\default\0.0.0\ecd7a3bf82a6c8c09551b39023d203551b61c1c3 (last modified on Sun Mar 30 19:45:26 2025).


Max length of source sentence: 432
Max length of target sentence: 435


<All keys matched successfully>

In [9]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

--------------------------------------------------------------------------------
Source: Just as it had once brought Banita face to face with a prime minister and then relegated her to angst-ridden anonymity , it brought her current plight to the notice of Bhakta Charan Das , a former Union minister and president of the Orissa Nava Nirman Manch .
Expected: प्रधानमंत्री से रू-ब-रू होने और उसके बाद दुखभरी गुमनामी में खो जाने के बाद उसकी वर्तमान दशा की खबर पूर्व केंद्रीय मंत्री और ओड़ीसा नवनिर्माण मंच के अध्यक्ष भक्तचरण दास को लगी .
Predicted: जिस तरह उसने एक बार प्रधानमंत्री के साथ प्रधानमंत्री का सामना किया और उसके बाद को देखते हुए उसने को प्रधानमंत्री कार्यालय में आमंत्रित किया . यह समिति दास ने निष्कर्ष पर ले जाया कि ओड़ीसा का वर्तमान सर्वोक्षण किया जाए .
--------------------------------------------------------------------------------
Source: Remember, the deck was stacked against you.
Expected: याद कीजिये, संभावनाएं आपके खिलाफ थीं|
Predicted: ज़रा देखिये , संभावनाएं आपके खिलाफ थीं |


In [5]:
model.parameters

<bound method Module.parameters of Transformer(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderBlock(
        (self_attention_block): MultiHeadAttentionBlock(
          (w_q): Linear(in_features=512, out_features=512, bias=False)
          (w_k): Linear(in_features=512, out_features=512, bias=False)
          (w_v): Linear(in_features=512, out_features=512, bias=False)
          (w_o): Linear(in_features=512, out_features=512, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward_block): FeedForwardBlock(
          (linear_1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (residual_connections): ModuleList(
          (0-1): 2 x ResidualConnection(
            (dropout): Dropout(p=0.1, inplace=False)
            (norm): LayerNormalization()
          )
        )
      )
 

In [6]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")

Total parameters: 90213680
